# 03 - Entrenamiento de Modelo Open Source con YOLOv8

Este notebook entrena un modelo de detección de EPP usando **YOLOv8**, la solución open source gratuita.

## Ventajas de YOLOv8:
- ✅ **Completamente gratis** y open source
- ✅ **State-of-the-art** en precisión y velocidad
- ✅ **Fácil de usar** con Ultralytics
- ✅ **Portabilidad total** - funciona en CPU o GPU
- ✅ **Sin costos cloud** - entrena localmente o en Colab gratis

## Requerimientos:
- Dataset anotado en formato YOLO (del notebook 01)
- GPU recomendada pero no obligatoria (Google Colab gratis tiene GPU)
- ~500-1000 imágenes anotadas para buenos resultados

In [ ]:
# Instalación de YOLOv8
!pip install ultralytics roboflow

In [ ]:
import os
import yaml
from pathlib import Path
from ultralytics import YOLO
import torch
from IPython.display import Image, display
import pandas as pd
import matplotlib.pyplot as plt

# Verificar disponibilidad de GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Dispositivo de entrenamiento: {device.upper()}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("   ⚠️ GPU no disponible. El entrenamiento será más lento pero funcionará.")
    print("   💡 Tip: Usa Google Colab (gratis) para GPU: https://colab.research.google.com")

In [ ]:
# Configuración de rutas
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
YOLO_DIR = DATA_DIR / 'processed' / 'yolo'
MODELS_DIR = DATA_DIR / 'models'
MODELS_DIR.mkdir(exist_ok=True)

# Cargar configuración del dataset
with open(YOLO_DIR / 'data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

print("📂 Dataset configurado:")
print(f"   Path: {data_config['path']}")
print(f"   Clases: {data_config['nc']}")
print(f"   Nombres: {data_config['names']}")

## Opción Rápida: Usar Dataset Pre-entrenado de Roboflow

Si aún no tienes tu dataset personalizado, puedes empezar con uno público:

In [ ]:
# Descomentar para usar dataset de Roboflow
# from roboflow import Roboflow
# from dotenv import load_dotenv

# load_dotenv('../.env')
# ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')

# if ROBOFLOW_API_KEY:
#     rf = Roboflow(api_key=ROBOFLOW_API_KEY)
#     # Ejemplo: usar un dataset público de PPE
#     # Busca uno en: https://universe.roboflow.com/search?q=ppe
#     # project = rf.workspace("roboflow-universe").project("ppe-detection-example")
#     # dataset = project.version(1).download("yolov8", location=str(DATA_DIR / 'roboflow'))
#     print("Dataset descargado desde Roboflow")
# else:
#     print("Configura ROBOFLOW_API_KEY en .env para descargar datasets")

## Paso 1: Seleccionar Modelo Base

YOLOv8 ofrece varios tamaños de modelo:

| Modelo | Tamaño | Velocidad | Precisión | Uso Recomendado |
|--------|--------|-----------|-----------|----------------|
| YOLOv8n | ~6 MB | ⚡⚡⚡ | ⭐⭐ | Edge devices, CPU |
| YOLOv8s | ~22 MB | ⚡⚡ | ⭐⭐⭐ | Balance óptimo |
| YOLOv8m | ~52 MB | ⚡ | ⭐⭐⭐⭐ | Mejor precisión |
| YOLOv8l | ~87 MB | ⚡ | ⭐⭐⭐⭐⭐ | Máxima precisión |

**Recomendación para tu caso**: YOLOv8s (balance perfecto)

In [ ]:
# Seleccionar modelo base
MODEL_SIZE = 'yolov8s'  # Cambiar a 'n', 's', 'm', 'l' según necesidad

# Cargar modelo pre-entrenado en COCO (transfer learning)
model = YOLO(f'{MODEL_SIZE}.pt')

print(f"✅ Modelo {MODEL_SIZE} cargado")
print(f"   El modelo viene pre-entrenado en COCO dataset (80 clases)")
print(f"   Lo adaptaremos a nuestras {data_config['nc']} clases de EPP")

## Paso 2: Configurar Hiperparámetros de Entrenamiento

In [ ]:
# Hiperparámetros de entrenamiento
EPOCHS = 100  # Número de épocas (100-200 recomendado)
BATCH_SIZE = 16  # Ajustar según VRAM disponible (16 para 8GB, 8 para 4GB)
IMG_SIZE = 640  # Tamaño de imagen (640 es estándar)
PATIENCE = 20  # Early stopping (parar si no mejora en 20 épocas)

# Configuración de entrenamiento
train_config = {
    'data': str(YOLO_DIR / 'data.yaml'),
    'epochs': EPOCHS,
    'batch': BATCH_SIZE,
    'imgsz': IMG_SIZE,
    'device': device,
    'patience': PATIENCE,
    'save': True,
    'project': str(MODELS_DIR / 'yolo_training'),
    'name': 'ppe_detector_v1',
    'exist_ok': True,
    
    # Data augmentation (ya incluido en YOLOv8)
    'hsv_h': 0.015,  # Variación de hue
    'hsv_s': 0.7,    # Variación de saturación
    'hsv_v': 0.4,    # Variación de brillo
    'degrees': 10,   # Rotación aleatoria
    'translate': 0.1,  # Traslación
    'scale': 0.5,    # Escala
    'flipud': 0.0,   # Flip vertical (no recomendado para EPP)
    'fliplr': 0.5,   # Flip horizontal
    'mosaic': 1.0,   # Mosaic augmentation
    'mixup': 0.1,    # Mixup augmentation
}

print("⚙️ Configuración de entrenamiento:")
for key, value in train_config.items():
    print(f"   {key}: {value}")

## Paso 3: Entrenar el Modelo

⚠️ **Importante**: 
- En CPU: ~1-2 horas por época (total: 100-200 horas)
- En GPU: ~1-5 minutos por época (total: 2-8 horas)
- Puedes usar Google Colab gratis con GPU T4

In [ ]:
# Entrenar modelo
print("🚀 Iniciando entrenamiento...")
print(f"   Esto puede tomar {'varias horas' if device == 'cpu' else '2-8 horas con GPU'}")
print("   Puedes detener con Ctrl+C y retomar después\n")

results = model.train(**train_config)

## Paso 4: Evaluar el Modelo en Validation Set

In [ ]:
# Validar el modelo
metrics = model.val()

print("\n📊 Métricas de Validación:")
print(f"   mAP50: {metrics.box.map50:.3f}")  # mAP @ IoU=0.5
print(f"   mAP50-95: {metrics.box.map:.3f}")  # mAP @ IoU=0.5:0.95
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

# Métricas por clase
print("\n📋 Métricas por Clase:")
for i, class_name in enumerate(data_config['names']):
    print(f"   {class_name}: mAP50={metrics.box.maps[i]:.3f}")

## Paso 5: Visualizar Resultados del Entrenamiento

In [ ]:
# Visualizar curvas de entrenamiento
training_results_dir = MODELS_DIR / 'yolo_training' / 'ppe_detector_v1'

# Mostrar gráficos de resultados
results_img = training_results_dir / 'results.png'
if results_img.exists():
    display(Image(filename=str(results_img)))
else:
    print("Gráficos de resultados se generarán al finalizar el entrenamiento")

# Matriz de confusión
confusion_matrix = training_results_dir / 'confusion_matrix.png'
if confusion_matrix.exists():
    print("\n📊 Matriz de Confusión:")
    display(Image(filename=str(confusion_matrix)))

## Paso 6: Probar el Modelo en Imágenes de Test

In [ ]:
# Cargar el mejor modelo entrenado
best_model_path = training_results_dir / 'weights' / 'best.pt'
best_model = YOLO(str(best_model_path))

print(f"✅ Mejor modelo cargado desde: {best_model_path}")

In [ ]:
# Hacer predicciones en imágenes de test
test_images_dir = YOLO_DIR / 'images' / 'test'

# Obtener algunas imágenes de prueba
test_images = list(test_images_dir.glob('*.jpg'))[:5]  # Primeras 5 imágenes

if test_images:
    for img_path in test_images:
        # Hacer predicción
        results = best_model.predict(
            source=str(img_path),
            conf=0.5,  # Umbral de confianza
            save=True,
            project=str(MODELS_DIR / 'predictions'),
            name='test_results',
            exist_ok=True
        )
        
        # Mostrar resultados
        print(f"\n🖼️ Predicciones para {img_path.name}:")
        for r in results:
            for box in r.boxes:
                class_id = int(box.cls)
                confidence = float(box.conf)
                class_name = data_config['names'][class_id]
                print(f"   {class_name}: {confidence:.2%}")
else:
    print("⚠️ No se encontraron imágenes de test")
    print("   Añade imágenes al directorio: {test_images_dir}")

## Paso 7: Exportar Modelo para Producción

Exporta el modelo a diferentes formatos según tu plataforma de deployment:

In [ ]:
# Exportar a diferentes formatos
export_formats = {
    'onnx': 'Formato universal - compatible con Azure, OpenCV',
    'torchscript': 'Para PyTorch en producción',
    'engine': 'TensorRT (máximo rendimiento con GPU NVIDIA)',
    'coreml': 'Para dispositivos Apple/iOS',
    'tflite': 'Para dispositivos móviles Android/Edge',
}

print("📦 Exportando modelo a diferentes formatos...\n")

# ONNX (recomendado para máxima compatibilidad)
onnx_path = best_model.export(format='onnx')
print(f"✅ ONNX exportado: {onnx_path}")

# TorchScript (para Python/PyTorch)
# torchscript_path = best_model.export(format='torchscript')
# print(f"✅ TorchScript exportado: {torchscript_path}")

print("\n💡 Tip: El formato ONNX es el más versátil para deployment en producción")

## Paso 8: Guardar Metadatos del Modelo

In [ ]:
# Guardar información del modelo
model_info = {
    'model_type': 'YOLOv8',
    'model_size': MODEL_SIZE,
    'num_classes': data_config['nc'],
    'class_names': data_config['names'],
    'image_size': IMG_SIZE,
    'epochs_trained': EPOCHS,
    'best_model_path': str(best_model_path),
    'onnx_model_path': str(onnx_path) if 'onnx_path' in locals() else None,
    'metrics': {
        'mAP50': float(metrics.box.map50),
        'mAP50-95': float(metrics.box.map),
        'precision': float(metrics.box.mp),
        'recall': float(metrics.box.mr),
    },
    'device_trained': device,
}

# Guardar como YAML
model_info_path = MODELS_DIR / 'yolo_ppe_model_info.yaml'
with open(model_info_path, 'w') as f:
    yaml.dump(model_info, f, default_flow_style=False)

print(f"✅ Información del modelo guardada en: {model_info_path}")
print("\n📋 Resumen del Modelo:")
for key, value in model_info.items():
    if key != 'class_names':  # Skip para no llenar la pantalla
        print(f"   {key}: {value}")

## Paso 9: Benchmark de Velocidad

In [ ]:
import time

# Benchmark de velocidad de inferencia
if test_images:
    test_img = test_images[0]
    
    # Warm-up
    for _ in range(3):
        _ = best_model.predict(str(test_img), verbose=False)
    
    # Medir velocidad
    num_iterations = 100
    start_time = time.time()
    
    for _ in range(num_iterations):
        _ = best_model.predict(str(test_img), verbose=False)
    
    end_time = time.time()
    avg_time = (end_time - start_time) / num_iterations
    fps = 1 / avg_time
    
    print("\n⚡ Benchmark de Velocidad:")
    print(f"   Tiempo promedio por imagen: {avg_time*1000:.1f} ms")
    print(f"   FPS (frames por segundo): {fps:.1f}")
    print(f"   Dispositivo: {device.upper()}")
    
    if fps >= 30:
        print("   ✅ Velocidad excelente para tiempo real")
    elif fps >= 10:
        print("   ✅ Velocidad adecuada para monitoreo de seguridad")
    else:
        print("   ⚠️ Velocidad lenta - considera usar GPU o modelo más pequeño")
else:
    print("No hay imágenes de test para benchmark")

## Resumen y Próximos Pasos

### ✅ Has completado:
1. ✅ Entrenamiento de modelo YOLOv8 personalizado
2. ✅ Evaluación de métricas (mAP, Precision, Recall)
3. ✅ Exportación a formato ONNX para producción
4. ✅ Benchmark de velocidad de inferencia

### 📊 Métricas Objetivo:
- **mAP50 > 0.85**: Excelente
- **Recall > 0.90**: Crítico para seguridad (detectar cuando falta EPP)
- **Precision > 0.80**: Reducir falsas alarmas

### 💡 Mejorando el Modelo:
Si tus métricas son bajas:
1. **Más datos**: Añade más imágenes anotadas (objetivo: 1000+)
2. **Mejor calidad**: Asegura anotaciones precisas
3. **Balance de clases**: Equilibra número de ejemplos por clase
4. **Más épocas**: Aumenta a 200-300 épocas
5. **Modelo más grande**: Prueba YOLOv8m o YOLOv8l

### ➡️ Siguiente notebook:
- **04_model_evaluation.ipynb**: Comparar Azure vs YOLOv8
- **05_testing_inference.ipynb**: Pruebas con cámaras en tiempo real

### 🚀 Deployment:
El modelo está listo para:
- Integración en la aplicación de producción
- Deployment en Docker container
- Uso con cámaras IP en tiempo real

### 💰 Costos:
- **Entrenamiento**: $0 (usando Colab gratis con GPU)
- **Inferencia**: $0 (modelo local)
- **Total**: $0 ✅